In [ ]:
'''
Neural style transfer -
    This code takes a base image and a style image and produces a
    combination image which has content of the base image and style of the
    style image(https://arxiv.org/pdf/1508.06576). This is achieved by
    minimising two losses- content loss(measure error between base and
    combination image pixel values) and style loss. The loss is minimised with
    bfgs algorithm and we update pixel values of combination image instead of
    weights of the network
'''

from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
from keras.applications import vgg19
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
import time
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# paths for base_image and style_image
base_image_path = './image1.jpg'
style_reference_image_path = './image2.jpg'

In [ ]:
from PIL import Image
img1 = Image.open(base_image_path)
img2 = Image.open(style_reference_image_path)

In [ ]:
fig = plt.figure(figsize=(15,8))
a = fig.add_subplot(1,2,1)
a.imshow(img1)
a.set_title('Base Image')
a = fig.add_subplot(1,2,2)
a.imshow(img2)
a.set_title('Style image')


In [ ]:
# weights for different loss funstions
total_variation_weight = 1.0
style_weight = 5.0
content_weight = 0.025

In [ ]:
# dimensions of the generated picture.
width, height = load_img(base_image_path).size
new_width = 512
new_height = int(width * new_width / height)

In [ ]:
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(new_width, new_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x = x.reshape((new_height, new_width, 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    # make sure values are between 0 and 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
#define various losses
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = new_height * new_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


def content_loss(base, combination):
    # MSE difference for base and combination image
    return K.sum(K.square(combination - base))



def total_variation_loss(x):
   
    a = K.square(x[:, :new_width - 1, :new_height - 1, :] - x[:, 1:, :new_height - 1, :])
    b = K.square(x[:, :new_width - 1, :new_height - 1, :] - x[:, :new_width - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [ ]:
def eval_loss_and_grads(x):
    x = x.reshape((1, new_width, new_height, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

In [ ]:
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(
    preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, new_width, new_height, 3))

# concatenate all three image tensors
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)
# define vgg19 model instance
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [ ]:
loss = K.variable(0.)
# get feature vector from block5 conv layer 2
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

In [ ]:
# style features from layers defined in Gatys et al. (2015)
# https://arxiv.org/pdf/1508.06576
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

In [ ]:
# get the gradient of the losses wrt combination image
grads = K.gradients(loss, combination_image)
outputs = [loss]
outputs += grads
f_outputs = K.function([combination_image], outputs)

In [ ]:
'''
We then introduce an Evaluator class that computes loss and gradients in one
pass while retrieving them via two separate functions, loss and grads.
This is done because scipy.optimize requires separate functions for loss and
gradients, but computing them separately would be inefficient.

'''


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


evaluator = Evaluator()

In [ ]:
# defining the combination_image to be the content image produces better
# results and leads to faster convergence than using a random image at the
# start
x = preprocess_image(base_image_path)

In [ ]:
iterations = 30

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    # using l_bfgs to minimise loss instead of sgd (to make it converge faster)
    # https://docs.scipy.org/doc/scipy/reference/generated/
    #                                         scipy.optimize.fmin_l_bfgs_b.html
    x, min_val, _ = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                  fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    img = deprocess_image(x.copy())
    fname = 'img_at_iteration_%d.png' % i
    # save image at each iteration
    imsave(fname, img)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))
